# Importing the data

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import contextily as ctx

In [2]:
df = pd.read_csv("../data/data_mmda_traffic_spatial.csv")

In [3]:
df.head(1)

,Date,Time,City,Location,Latitude,Longitude,High_Accuracy,Direction,Type,Lanes_Blocked,Involved,Tweet,Source
0,2018-08-20,7:55 AM,Pasig City,ORTIGAS EMERALD,14.586343,121.061481,1,EB,VEHICULAR ACCIDENT,1.0,TAXI AND MC,MMDA ALERT: Vehicular accident at Ortigas Emer...,https://twitter.com/mmda/status/10313302019705...


# Information about the data
Let's find out more about the data including the number of rows, columns, missing data, etc.

In [4]:
df.shape

(17312, 13)

In [5]:
df.dtypes

Date              object
Time              object
City              object
Location          object
Latitude         float64
Longitude        float64
High_Accuracy      int64
Direction         object
Type              object
Lanes_Blocked    float64
Involved          object
Tweet             object
Source            object
dtype: object

In [6]:
df.describe()

,Latitude,Longitude,High_Accuracy,Lanes_Blocked
count,17312.000000,17312.000000,17312.000000,16625.000000
mean,14.559448,120.666794,0.955638,1.097624
std,0.822927,6.812422,0.205905,0.302237
min,0.000000,0.000000,0.000000,1.000000
25%,14.577625,121.042734,1.000000,1.000000
50%,14.603015,121.053801,1.000000,1.000000
75%,14.632910,121.069619,1.000000,1.000000
max,14.735495,121.119655,1.000000,6.000000


In [7]:
df.isna().sum()

Date               0
Time             122
City             187
Location          23
Latitude           0
Longitude          0
High_Accuracy      0
Direction        857
Type              57
Lanes_Blocked    687
Involved         432
Tweet              0
Source             0
dtype: int64

### Total rows with missing values percentage %

In [8]:
df.isna().any(axis=1).sum() / df.shape[0] * 100

11.541127541589649

# Cleaning the data

### Drop rows with null values
Since only 12% of the rows on our dataset have null values, we can drop them. This is not always the best approach but it is more convenient for now.

In [9]:
df.dropna(inplace=True)

In [10]:
df.shape[0]

15314

### Only select vehicular accidents
This dataset includes data on many types of traffic incidents. We want to include rows that corresponds to vehicular accidents only. For this, we can search through the Type column and only include those that have the keywords "Accident", "Collision", and "Hit".

In [11]:
# Now, we subset the dataframe to only include incident types that have these keywords on them
df = df[df['Type'].str.lower().str.contains(r"accident|accidentat|collision|hit", regex=True)]

df.reset_index(inplace=True, drop=True)

In [12]:
print(f'No. of accidents: {df.shape[0]}')

No. of accidents: 11595


We see that only 76% of the whole dataset (without null values) involves vehicular accidents.

### Categorize each accident
Right now, the "Type" column is messy and have many different values. Using the keywords we used earlier, we can categorize the accident types into three:
- Accident (involving at most 2 vehicles)
- Multiple collision
- Hit and Run

In [13]:
def categorize_accident(val):
    """
    Categorize the accident based on keywords found on Type column.
    """
    type = val.upper()

    if 'HIT AND RUN' in type:
        return 'HIT AND RUN'
    elif 'MULTIPLE COLLISION' in type:
        return 'MULTIPLE COLLISION'
    elif 'ACCIDENT' or 'ACCIDENTAT' in type:
        return 'ACCIDENT'
    else:
        return 'OTHERS'

In [14]:
df['Type'] = df['Type'].apply(lambda val: categorize_accident(val))
df['Type'].value_counts()

ACCIDENT              11028
MULTIPLE COLLISION      528
HIT AND RUN              39
Name: Type, dtype: int64

### Cleaning the "Direction" column
Since the values for each column are extracted from a tweet, some are inconsistent. For example, we have 2 unique values for the EB category: "EB" and "EB.", so we need to fix it.

In [15]:
df['Direction'].unique()

array(['EB', 'SB', 'NB', 'EB.', 'WB', 'DAR', 'CLARA'], dtype=object)

In [16]:
# Replace all 'EB.' with 'EB'
df.replace('EB.', 'EB', inplace=True)

In [17]:
# Remove rows with 'DAR', 'CLARA'
df.drop(df[df['Direction'].isin(['DAR', 'CLARA'])].index, inplace=True)

Now, we only have 4 possible values on the "Direction" column.
- EB: East Bound
- SB: South Bound
- NB: North Bound
- WB: West Bound

In [18]:
df['Direction'].unique()

array(['EB', 'SB', 'NB', 'WB'], dtype=object)

### Cleaning the "City" column
We have abnormal values on the City column such as "ParaÃ±aque" which meant to be "Parañaque".

In [19]:
df['City'].unique()

array(['Pasig City', 'Makati City', 'San Juan', 'Quezon City',
       'Mandaluyong', 'Marikina', 'Pasay City', 'Manila', 'Kalookan City',
       'ParaÃ±aque', 'Taguig', 'Navotas', 'Valenzuela', 'Malabon',
       'Parañaque'], dtype=object)

In [20]:
df.replace('ParaÃ±aque', 'Parañaque', inplace=True)

### Cleaning the "Time" column
There are values that have both "AM PM" in them. Upon searching the source of the tweet, those accidents all occured in the morning.

In [21]:
df[df['Time'].str.split().str.len() > 2].loc[:, 'Time']

9126     8:20 AM AM
9127     9:37 AM AM
11525    6:26 AM PM
11526    8:05 AM PM
Name: Time, dtype: object

In [22]:
def remove_pm(val):
    """
    Remove the last word "PM" on the string.
    """
    split = val.split()
    if len(split) > 2:
        split.pop()
        return f'{split[0]} {split[1]}'
    else:
        return val

In [23]:
df['Time'] = df['Time'].apply(lambda val: remove_pm(val))

In [24]:
df[df['Time'].str.split().str.len() > 2].loc[:, 'Time']

Series([], Name: Time, dtype: object)

Now, there are no times with both "AM" and "PM".

There are still other inconsistencies with the Time column such as some being on military time

In [25]:
# Fix military time format (there is only one row with this behavior)
df['Time'].replace('22:55 PM', '10:55 PM', inplace=True)

...and others not having "AM" or "PM" or is incomplete.

In [26]:
# Remove excess whitespace on Time column
df['Time'] = df['Time'].apply(lambda val: val.strip())

In [27]:
# Show Time values without "AM" or "PM" or incomplete
incomplete = df[df['Time'].apply(lambda val: val[-2:] not in ['AM', 'PM'])]

incomplete.loc[:, ['Time', 'Tweet', 'Source']].sort_values('Time').values

array([['12:00',
        'MMDA ALERT: Vehicular accident at EDSA Santolan SB involving 2 buses as of 12:00 NN. 1 lane occupied. MMDA on site. #mmda',
        'https://twitter.com/mmda/status/1114388117165731840'],
       ['12:00',
        'MMDA ALERT: Vehicular accident at EDSA Timog Eb involving PUJ and MC as of 12:00 NN. 1 lane occupied. MMDA on site. #mmda',
        'https://twitter.com/mmda/status/1120902250887389185'],
       ['12:00',
        'MMDA ALERT: Vehicular accident at EDSA Megamall NB involving bus and AUV as of 12:00 NN. 1 lane occupied. MMDA on site. #mmda',
        'https://twitter.com/mmda/status/1129599110246363137'],
       ['12:31',
        'MMDA ALERT: Vehicular accident at Mindanao Ave Tandang Sora SB involving van and AUV as of 12:31 pM. 1 lane occupied. MMDA on site. #mmda',
        'https://twitter.com/mmda/status/1176718107122925568'],
       ['2:25',
        'MMDA ALERT: Vehicular accident at EDSA Santolan Flyover SB involving Bus and Pick-up as of 2:25 pm.

Upon looking at the source of the tweet, all of these occured in the noon onwards. So it is safe to classify all of them as "PM".

In [28]:
# Remove extra M's
df.loc[incomplete.index, 'Time'] = df.loc[incomplete.index, 'Time'].apply(lambda val: val[:-1] if val[-1] == 'M' else val)

df.loc[incomplete.index, 'Time'].values

array(['12:00', '12:00', '12:00', '7:21', '7:57', '8:26', '12:31', '5:25',
       '8:36', '2:25', '3:32', '4:31'], dtype=object)

In [29]:
# Add "PM" at the end
df.loc[incomplete.index, 'Time'] = df.loc[incomplete.index, 'Time'].apply(lambda val: f'{val} PM')

df.loc[incomplete.index, 'Time'].values

array(['12:00 PM', '12:00 PM', '12:00 PM', '7:21 PM', '7:57 PM',
       '8:26 PM', '12:31 PM', '5:25 PM', '8:36 PM', '2:25 PM', '3:32 PM',
       '4:31 PM'], dtype=object)

...and others not having a whitespace before "AM" or "PM".

In [30]:
def no_whitespace(val):
    """
    Determine if the time format has no whitespace before "AM" or "PM".
    """
    if len(val.split()) < 2:
        return True
    else:
        return False

In [31]:
# Show all values without whitespace before "AM" or "PM"
df[df['Time'].apply(no_whitespace)].loc[:, 'Time'].values

array(['3:20PM', '8:55PM', '9:00PM', '8:00PM', '8:05PM', '6:07PM',
       '6:17PM', '6:48PM', '2:27PM', '4:49PM', '4:17PM', '4:10PM',
       '5:21PM', '6:30PM', '7:03PM', '3:23PM', '3:07PM', '3:07PM',
       '7:20PM', '3:46PM', '4:09PM', '4:13PM', '4:08PM', '4:33PM',
       '3:29PM'], dtype=object)

In [32]:
def fix_whitespace(val):
    """
    Add whitespace before the "AM" or "PM".
    """
    val = val.strip()

    if no_whitespace(val):
        ind = val[-2:]
        time = val[:-2]
        return f'{time} {ind}'
    else:
        return val.strip()

In [33]:
# Fix all values without whitespace before "AM" or "PM"
df['Time'] = df['Time'].apply(fix_whitespace)

### Concatenate Date and Time column

We can concatenate the Date and Time column and convert its datatype to datetime later.

In [34]:
# Concatenate Date and Time column to create new column Datetime
df['Datetime'] = df['Date'].str.cat(df['Time'], sep=' ')
date_time = df.pop('Datetime')
df.insert(0, 'Datetime', date_time)

# Drop the Date and Time column
df.drop(columns=['Date', 'Time'], inplace=True)

df.head(1)

,Datetime,City,Location,Latitude,Longitude,High_Accuracy,Direction,Type,Lanes_Blocked,Involved,Tweet,Source
0,2018-08-20 7:55 AM,Pasig City,ORTIGAS EMERALD,14.586343,121.061481,1,EB,ACCIDENT,1.0,TAXI AND MC,MMDA ALERT: Vehicular accident at Ortigas Emer...,https://twitter.com/mmda/status/10313302019705...


### Drop unnecessary columns
This data is extracted from a series of tweets from the Metro Manila Development Authority (MMDA) thus, some columns not necessary for clustering can be dropped like the High_Accuracy, Tweet, and Source column.

In [35]:
df.drop(columns=['High_Accuracy', 'Tweet', 'Source'], inplace=True)

In [36]:
df.head(3)

,Datetime,City,Location,Latitude,Longitude,Direction,Type,Lanes_Blocked,Involved
0,2018-08-20 7:55 AM,Pasig City,ORTIGAS EMERALD,14.586343,121.061481,EB,ACCIDENT,1.0,TAXI AND MC
1,2018-08-20 9:13 AM,Makati City,EDSA ROCKWELL,14.559818,121.040737,SB,ACCIDENT,1.0,SUV AND L300
2,2018-08-20 10:27 AM,San Juan,ORTIGAS CLUB FILIPINO,14.601846,121.046754,EB,ACCIDENT,1.0,2 CARS


# Export the dataframe
Now that we've cleaned the data, let us export it as a .csv file so it will be easier to access across notebooks.

In [37]:
df.to_csv('../data/data_mmda_traffic_spatial_cleaned.csv', index=False)

In [38]:
cleaned_df = pd.read_csv('../data/data_mmda_traffic_spatial_cleaned.csv')

cleaned_df.head()

,Datetime,City,Location,Latitude,Longitude,Direction,Type,Lanes_Blocked,Involved
0,2018-08-20 7:55 AM,Pasig City,ORTIGAS EMERALD,14.586343,121.061481,EB,ACCIDENT,1.0,TAXI AND MC
1,2018-08-20 9:13 AM,Makati City,EDSA ROCKWELL,14.559818,121.040737,SB,ACCIDENT,1.0,SUV AND L300
2,2018-08-20 10:27 AM,San Juan,ORTIGAS CLUB FILIPINO,14.601846,121.046754,EB,ACCIDENT,1.0,2 CARS
3,2018-08-20 12:33 PM,Quezon City,EDSA ORTIGAS ROBINSONS,14.592515,121.058225,NB,ACCIDENT,1.0,CAR AND BUS
4,2018-08-20 12:39 PM,Mandaluyong,EDSA LIGHT MALL,14.574014,121.048410,NB,MULTIPLE COLLISION,2.0,3 CARS


Now, our data is ready for preprocessing for machine learning models.